# Tomato Leaf Disease Detection: InceptionV3 Optimized for Ryzen 5 5600G

### 1. Environment Setup


In [1]:
!pip install tensorflow-cpu opencv-python matplotlib seaborn numpy
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision
set_global_policy('mixed_float16')

# Configure TensorFlow for CPU (12 threads)
tf.config.threading.set_intra_op_parallelism_threads(12)
tf.config.threading.set_inter_op_parallelism_threads(12)

  Using cached tensorflow_cpu-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow_cpu-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Obidur Rahman\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2. Dataset Configuration


In [2]:
# Global constants
batch_size = 8
TRAIN_PERCENT = 0.8
img_size = (150, 150)

dataset_dir = "dataset"

# Get class names
class_names = sorted([d for d in os.listdir(dataset_dir) 
                     if os.path.isdir(os.path.join(dataset_dir, d)) and not d.startswith('.')])
print(f"Detected classes: {class_names}")
print(f"Total number of classes: {len(class_names)}")

# Count total images
total_images = 0
for class_name in class_names:
    class_dir = os.path.join(dataset_dir, class_name)
    num_images = len([f for f in os.listdir(class_dir) 
                     if os.path.isfile(os.path.join(class_dir, f))])
    print(f"Class '{class_name}': {num_images} images")
    total_images += num_images
print(f"Total images in dataset: {total_images}")

Detected classes: ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Total number of classes: 10
Class 'Tomato_Bacterial_spot': 2127 images
Class 'Tomato_Early_blight': 1000 images
Class 'Tomato_Late_blight': 1909 images
Class 'Tomato_Leaf_Mold': 952 images
Class 'Tomato_Septoria_leaf_spot': 1771 images
Class 'Tomato_Spider_mites_Two_spotted_spider_mite': 1676 images
Class 'Tomato__Target_Spot': 1404 images
Class 'Tomato__Tomato_YellowLeaf__Curl_Virus': 3209 images
Class 'Tomato__Tomato_mosaic_virus': 373 images
Class 'Tomato_healthy': 1591 images
Total images in dataset: 16012


### 3. Data Loading & Preprocessing


In [3]:
def hybrid_segmentation(img):
    # Simplified segmentation
    img = img.numpy()  # Convert tensor to NumPy for OpenCV
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    lower_green = np.array([25, 40, 50])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

    # Fallback to full image if segmentation fails
    if np.sum(mask) < 0.2 * mask.size:
        mask = np.ones(img.shape[:2], np.uint8) * 255

    segmented = cv2.bitwise_and(img, img, mask=mask)
    return segmented

def process_image(file_path, label, img_size=(150, 150)):
    # Load and preprocess a single image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Adjust for PNG if needed
    img = tf.image.convert_image_dtype(img, tf.float32)  # Use float32
    img = tf.image.resize(img, img_size)
    img = tf.py_function(func=hybrid_segmentation, inp=[img], Tout=tf.float32)
    img.set_shape([img_size[0], img_size[1], 3])
    return img, label

# Data augmentation layers
augmentation = models.Sequential([
    layers.RandomRotation(0.0417),  # ~15 degrees (15/360)
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomZoom(0.1),
    layers.RandomFlip("horizontal")
])

def create_dataset(dataset_dir, class_names, img_size=(150, 150), train_percent=0.8):
    file_paths = []
    labels = []
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(dataset_dir, class_name)
        files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) 
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        file_paths.extend(files)
        labels.extend([class_idx] * len(files))
        print(f"Processing class {class_name}... ({len(files)} images)")

    # Convert to tensors
    file_paths = tf.constant(file_paths)
    labels = tf.constant(labels)

    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.shuffle(buffer_size=len(file_paths), seed=42)
    dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Split into train and test
    total_size = len(file_paths)
    train_size = int(train_percent * total_size)
    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size)

    # Apply augmentation to training dataset
    train_dataset = train_dataset.map(lambda x, y: (augmentation(x, training=True), y), 
                                     num_parallel_calls=tf.data.AUTOTUNE)

    # Batch and prefetch
    train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_dataset, test_dataset, labels

# Load datasets
train_dataset, test_dataset, y_labels = create_dataset(dataset_dir, class_names, img_size)
print(f"Dataset loaded: ~{total_images} images, {img_size} resolution")
print(f"Training set: ~{int(total_images * TRAIN_PERCENT)} images")
print(f"Testing set: ~{total_images - int(total_images * TRAIN_PERCENT)} images")

Processing class Tomato_Bacterial_spot... (2127 images)
Processing class Tomato_Early_blight... (1000 images)
Processing class Tomato_Late_blight... (1909 images)
Processing class Tomato_Leaf_Mold... (952 images)
Processing class Tomato_Septoria_leaf_spot... (1771 images)
Processing class Tomato_Spider_mites_Two_spotted_spider_mite... (1676 images)
Processing class Tomato__Target_Spot... (1404 images)
Processing class Tomato__Tomato_YellowLeaf__Curl_Virus... (3208 images)
Processing class Tomato__Tomato_mosaic_virus... (373 images)
Processing class Tomato_healthy... (1591 images)
Dataset loaded: ~16012 images, (150, 150) resolution
Training set: ~12809 images
Testing set: ~3203 images


### 4. Deep Learning Pipeline


In [4]:
def build_inceptionv3_model(input_shape=(150, 150, 3), num_classes=len(class_names)):
    base = InceptionV3(include_top=False, input_shape=input_shape, weights='imagenet')
    base.trainable = False
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize model
model = build_inceptionv3_model()

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_inceptionv3.h5', save_best_only=True)
]

# Train model
print("Training InceptionV3...")
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=test_dataset,
    callbacks=callbacks,
    verbose=1
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_dataset, verbose=0)
print(f"InceptionV3 Test Accuracy: {test_acc:.4f}")

Training InceptionV3...
Epoch 1/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.3490 - loss: 2.0892

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 311s 190ms/step - accuracy: 0.3491 - loss: 2.0890 - val_accuracy: 0.6781 - val_loss: 0.9592
Epoch 2/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.5931 - loss: 1.2275

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 291s 182ms/step - accuracy: 0.5931 - loss: 1.2275 - val_accuracy: 0.7393 - val_loss: 0.8007
Epoch 3/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.6496 - loss: 1.0649

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 293s 183ms/step - accuracy: 0.6496 - loss: 1.0649 - val_accuracy: 0.7590 - val_loss: 0.6940
Epoch 4/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.6711 - loss: 0.9899

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 292s 182ms/step - accuracy: 0.6711 - loss: 0.9899 - val_accuracy: 0.7827 - val_loss: 0.6659
Epoch 5/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.6907 - loss: 0.9294

...

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 295s 184ms/step - accuracy: 0.7153 - loss: 0.8656 - val_accuracy: 0.8261 - val_loss: 0.5335
Epoch 9/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 326s 204ms/step - accuracy: 0.7187 - loss: 0.8206 - val_accuracy: 0.8139 - val_loss: 0.5393
Epoch 10/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 308s 193ms/step - accuracy: 0.7216 - loss: 0.8268 - val_accuracy: 0.8133 - val_loss: 0.5556
Epoch 11/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.7336 - loss: 0.7850

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 293s 183ms/step - accuracy: 0.7336 - loss: 0.7850 - val_accuracy: 0.8186 - val_loss: 0.5286
Epoch 12/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.7322 - loss: 0.7886

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 294s 184ms/step - accuracy: 0.7322 - loss: 0.7886 - val_accuracy: 0.8308 - val_loss: 0.5139
Epoch 13/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.7311 - loss: 0.7863

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 311s 194ms/step - accuracy: 0.7311 - loss: 0.7863 - val_accuracy: 0.8320 - val_loss: 0.5091
Epoch 14/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.7464 - loss: 0.7594

1601/1601 ━━━━━━━━━━━━━━━━━━━━ 327s 204ms/step - accuracy: 0.7464 - loss: 0.7594 - val_accuracy: 0.8345 - val_loss: 0.4849
Epoch 15/15
1601/1601 ━━━━━━━━━━━━━━━━━━━━ 304s 190ms/step - accuracy: 0.7409 - loss: 0.7584 - val_accuracy: 0.8336 - val_loss: 0.5144
InceptionV3 Test Accuracy: 0.8336


### 5. Evaluation


In [7]:
%matplotlib inline
# Plot accuracy and loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.close()

# Confusion matrix
y_pred = []
y_true = []
for images, labels in test_dataset:
    preds = model.predict(images, verbose=0)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())
y_pred = np.array(y_pred)
y_true = np.array(y_true)

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - InceptionV3')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.close()

### 6. Model Saving


In [ ]:
model.save('best_inceptionv3.h5')
print("InceptionV3 model saved as best_inceptionv3.h5")